In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

In [4]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


Average target value in full training set vs. first 50K rows

In [5]:
train_subset = train[:50000]
print('Average target value, full training set: ', train['target'].mean())
print('Average target value, first 50K rows: ', train_subset['target'].mean())

Average target value, full training set:  0.10301729306264126
Average target value, first 50K rows:  0.0863802290186301


Average values of all numeric columns, first in full training set and then in first 50,000 instances of training set.

In [6]:
train.mean(axis=0, skipna=True, numeric_only=True)

id                                     3.738434e+06
target                                 1.030173e-01
severe_toxicity                        4.582099e-03
obscene                                1.387721e-02
identity_attack                        2.263571e-02
insult                                 8.115273e-02
threat                                 9.311271e-03
asian                                  1.196355e-02
atheist                                3.205019e-03
bisexual                               1.884285e-03
black                                  3.439263e-02
buddhist                               1.410497e-03
christian                              9.526806e-02
female                                 1.276703e-01
heterosexual                           3.237040e-03
hindu                                  1.457378e-03
homosexual_gay_or_lesbian              2.561058e-02
intellectual_or_learning_disability    1.088100e-03
jewish                                 1.786259e-02
latino      

In [7]:
train_subset.mean(axis=0, skipna=True, numeric_only=True)

id                                     272378.677620
target                                      0.086380
severe_toxicity                             0.004108
obscene                                     0.014913
identity_attack                             0.015083
insult                                      0.067001
threat                                      0.009800
asian                                       0.004941
atheist                                     0.004025
bisexual                                    0.003218
black                                       0.029553
buddhist                                    0.000858
christian                                   0.052283
female                                      0.127134
heterosexual                                0.003788
hindu                                       0.000766
homosexual_gay_or_lesbian                   0.033891
intellectual_or_learning_disability         0.001342
jewish                                      0.

What is the percentage difference between the average values in the full training set and the first 50,000 instances?

In [8]:
(train.mean(axis=0, skipna=True, numeric_only=True) - train_subset.mean(axis=0, skipna=True, numeric_only=True))/ train.mean(axis=0, skipna=True, numeric_only=True)

id                                     0.927141
target                                 0.161498
severe_toxicity                        0.103489
obscene                               -0.074617
identity_attack                        0.333679
insult                                 0.174384
threat                                -0.052492
asian                                  0.586997
atheist                               -0.255705
bisexual                              -0.707830
black                                  0.140713
buddhist                               0.391434
christian                              0.451206
female                                 0.004197
heterosexual                          -0.170285
hindu                                  0.474720
homosexual_gay_or_lesbian             -0.323310
intellectual_or_learning_disability   -0.232908
jewish                                 0.482149
latino                                 0.267209
male                                   0

What percentage of the examples in the full training set and in the training subset have any identity mentioned?

In [9]:
identity_cols = ['asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian',
                 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
                 'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
                 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity',
                 'other_religion', 'other_sexual_orientation', 'physical_disability',
                 'psychiatric_or_mental_illness', 'transgender', 'white']
train['identity_any'] = train[identity_cols].ge(0.5).any(1)
train_subset['identity_any'] = train_subset[identity_cols].ge(0.5).any(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
print('Percentage of instances in full training set with any identity marker: ',
      train['identity_any'].sum()/len(train['identity_any']))
print('Percentage of instances in first 50K instances with any identity marker: ',
      train_subset['identity_any'].sum()/len(train_subset['identity_any']))

Percentage of instances in full training set with any identity marker:  0.09435783328919359
Percentage of instances in first 50K instances with any identity marker:  0.06726


Comments with any mention of identity markers are more likely to be labeled as toxic than comments without identity markers, both in the full dataset and in the first 50K examples.

In [11]:
train.groupby('identity_any')[['target','severe_toxicity','obscene','identity_attack','insult','threat']].mean()

,target,severe_toxicity,obscene,identity_attack,insult,threat
identity_any,,,,,,
False,0.093249,0.003976,0.013386,0.012248,0.076484,0.008706
True,0.196773,0.010396,0.018587,0.122336,0.125963,0.015117


In [12]:
train_subset.groupby('identity_any')[['target','severe_toxicity','obscene','identity_attack','insult','threat']].mean()

,target,severe_toxicity,obscene,identity_attack,insult,threat
identity_any,,,,,,
False,0.078854,0.003681,0.014487,0.008217,0.062808,0.009402
True,0.190748,0.010028,0.020816,0.110300,0.125147,0.015324


How many non-toxic comments are there that mention identity markers after the first 50K examples?

In [13]:
train['toxic'] = train['target'].ge(0.5)
train_subset['toxic'] = train_subset['target'].ge(0.5)

train[50000:].groupby('toxic')['identity_any'].count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


toxic
False    1613843
True      141031
Name: identity_any, dtype: int64

In [14]:
train_subset.groupby('toxic')['identity_any'].count()

toxic
False    46697
True      3303
Name: identity_any, dtype: int64

Choosing columns to augment with nontoxic examples

In [15]:
train[50000:].groupby('toxic')[identity_cols].mean()

,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white
toxic,,,,,,,,,,,,,,,,,,,,,,,,
False,0.012090,0.003114,0.001741,0.026675,0.001425,0.098705,0.124071,0.002850,0.001483,0.020599,0.000883,0.017092,0.005729,0.104355,0.043947,0.001201,0.000854,0.008014,0.006744,0.001378,0.001316,0.010608,0.006089,0.046351
True,0.012504,0.003733,0.002711,0.095390,0.001416,0.077912,0.155746,0.006126,0.001410,0.062736,0.002624,0.025719,0.009570,0.143740,0.099786,0.001361,0.001154,0.010638,0.007492,0.002372,0.001552,0.022605,0.011580,0.141211


In [18]:
print('Number of toxic examples in full training set: %d (%.2f%%)' % \
      (len(train.loc[train['toxic']==True]), len(train.loc[train['toxic']==True])/len(train)*100.))
print('Number of toxic examples in first 50K rows: %d (%.2f%%)' % \
      (len(train_subset.loc[train_subset['toxic']==True]), len(train_subset.loc[train_subset['toxic']==True])/len(train_subset)*100.))
for col in identity_cols:
    num_identity_train = len(train.loc[train[col].ge(0.5)])
    num_identity_subset = len(train_subset.loc[train_subset[col].ge(0.5)])
    num_toxic_train = len(train.loc[train['toxic']==True])
    num_toxic_subset = len(train_subset.loc[train_subset['toxic']==True])
    num_toxic_identity_train = len(train.loc[(train[col].ge(0.5)) & (train['toxic']==True)])
    num_toxic_identity_subset = len(train_subset.loc[(train_subset[col].ge(0.5)) & (train_subset['toxic']==True)])
    total_train = len(train)
    total_subset = len(train_subset)
    print('%s, full train set: %d toxic (%.2f%% of rows, %.2f%% of toxic)' % \
          (col, num_toxic_identity_train, num_identity_train/total_train*100., num_toxic_identity_train/num_toxic_train*100.))
    print('%s, first 50K rows: %d toxic (%.2f%% of rows, %.2f%% of toxic)' % \
          (col, num_toxic_identity_subset, num_identity_subset/total_subset*100., num_toxic_identity_subset/num_toxic_subset*100.))

Number of toxic examples in full training set: 144334 (8.00%)
Number of toxic examples in first 50K rows: 3303 (6.61%)
asian, full train set: 566 toxic (0.25% of rows, 0.39% of toxic)
asian, first 50K rows: 4 toxic (0.09% of rows, 0.12% of toxic)
atheist, full train set: 185 toxic (0.08% of rows, 0.13% of toxic)
atheist, first 50K rows: 3 toxic (0.09% of rows, 0.09% of toxic)
bisexual, full train set: 62 toxic (0.02% of rows, 0.04% of toxic)
bisexual, first 50K rows: 0 toxic (0.02% of rows, 0.00% of toxic)
black, full train set: 4678 toxic (0.83% of rows, 3.24% of toxic)
black, first 50K rows: 72 toxic (0.62% of rows, 2.18% of toxic)
buddhist, full train set: 69 toxic (0.03% of rows, 0.05% of toxic)
buddhist, first 50K rows: 1 toxic (0.01% of rows, 0.03% of toxic)
christian, full train set: 3673 toxic (2.24% of rows, 2.54% of toxic)
christian, first 50K rows: 71 toxic (1.02% of rows, 2.15% of toxic)
female, full train set: 7311 toxic (2.96% of rows, 5.07% of toxic)
female, first 50K ro

In [19]:
cols_to_augment = ['asian', 'bisexual', 'black', 'christian','female', 'heterosexual', 
                   'homosexual_gay_or_lesbian', 'jewish', 'latino', 'male',
                   'muslim', 'other_race_or_ethnicity', 'psychiatric_or_mental_illness', 
                   'transgender', 'white']

train_after_50k = train[50000:]
data_for_augmentation = train_after_50k.loc[(train_after_50k[cols_to_augment].ge(0.5).any(1)) & (train_after_50k['toxic']==False)]
print(len(train_after_50k), len(data_for_augmentation))

1754874 138543


In [20]:
for col in cols_to_augment:
    num_non_toxic = len(train_after_50k.loc[(train_after_50k[col].ge(0.5)) & (train_after_50k['toxic']==False)])
    total = len(train_after_50k.loc[train_after_50k[col].ge(0.5)])
    print('Nontoxic examples after first 50K with identity %s: %d (%.2f%% of all %s)' % \
          (col, num_non_toxic, num_non_toxic/total*100., col))

Nontoxic examples after first 50K with identity asian: 3973 (87.61% of all asian)
Nontoxic examples after first 50K with identity bisexual: 214 (77.54% of all bisexual)
Nontoxic examples after first 50K with identity black: 9986 (68.43% of all black)
Nontoxic examples after first 50K with identity christian: 36309 (90.97% of all christian)
Nontoxic examples after first 50K with identity female: 45001 (86.30% of all female)
Nontoxic examples after first 50K with identity heterosexual: 967 (77.24% of all heterosexual)
Nontoxic examples after first 50K with identity homosexual_gay_or_lesbian: 7597 (71.39% of all homosexual_gay_or_lesbian)
Nontoxic examples after first 50K with identity jewish: 6329 (83.77% of all jewish)
Nontoxic examples after first 50K with identity latino: 1583 (80.52% of all latino)
Nontoxic examples after first 50K with identity male: 36922 (84.89% of all male)
Nontoxic examples after first 50K with identity muslim: 16112 (77.25% of all muslim)
Nontoxic examples afte

Still to do:
- Figure out how to choose number of examples from each identity to balance the percentages
- See if there are too few nontoxic examples of any identity to do the augmentation (there are only 214 for bisexual for example)